In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

from string import punctuation
import re

In [ ]:
faq =pd.read_csv('../input/mental-health-faq-for-chatbot/Mental_Health_FAQ.csv')
faq

In [ ]:
faq_quest = faq[['Question_ID', 'Questions']]
faq_answ = faq[['Question_ID', 'Answers']]

def to_lower(text):
    return text.lower()

def expand_contraction(text, contraction_dict):
    contraction_pattern= re.compile('({})'.format('|'.join(contraction_dict.keys())), flags= re.IGNORECASE | re.DOTALL)
    
    def expand_match(contraction):
        match= contraction.group(0)
        first_char= match[0]
        expanded_contraction= contraction_dict.get(match) \
            if contraction_dict.get(match) \
            else contraction_dict.get(match.lower())
        expanded_contraction= expanded_contraction
        return expanded_contraction
        
    expanded_text= contraction_pattern.sub(expand_match, text)
    expanded_text= re.sub("'","", expanded_text)
    return expanded_text

def main_contraction(text):
    text = expand_contraction(text, contractions_dict)
    return text


def remove_number(text):
    output = ''.join(c for c in text if not c.isdigit())
    return output
def remove_punct(text):
    return "".join(c for c in text if c not in punctuation)
def to_strip(text):
    return " ".join([c for c in text.split() if len(c)>2])
def remove_char(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A)
    return text
def remove_duplicate(text):
    text = re.sub("(.)\\1{2,}", "\\1", text)
    return text 

import nltk
from nltk.corpus import stopwords
stopwords.words('english')

def remove_stopwords(text):
    stop_words= stopwords.words('english')
    return ' '.join(c for c in nltk.word_tokenize(text) if c not in stop_words)

from nltk.stem import WordNetLemmatizer

wordnet_lemma = WordNetLemmatizer()

def lemma(text):
    lemmatize_words = [wordnet_lemma.lemmatize(word) for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    return ' '.join(lemmatize_words)

faq_quest['prep1']= faq_quest['Questions'].apply(to_lower)
faq_quest['prep2']= faq_quest['prep1'].apply(main_contraction)
faq_quest['prep3']= faq_quest['prep2'].apply(remove_number)
faq_quest['prep4']= faq_quest['prep3'].apply(remove_punct)
faq_quest['prep5']= faq_quest['prep4'].apply(to_strip)
faq_quest['prep6']= faq_quest['prep5'].apply(remove_char)
faq_quest['prep7']= faq_quest['prep6'].apply(remove_duplicate)
faq_quest['prep8']= faq_quest['prep7'].apply(remove_stopwords)
faq_quest['lemma']= faq_quest['prep8'].apply(lemma)
faq_quest.head(10)

faq_answ['prep1']= faq_answ['Answers'].apply(to_lower)
faq_answ['prep2']= faq_answ['prep1'].apply(main_contraction)
faq_answ['prep3']= faq_answ['prep2'].apply(remove_number)
faq_answ['prep4']= faq_answ['prep3'].apply(remove_punct)
faq_answ['prep5']= faq_answ['prep4'].apply(to_strip)
faq_answ['prep6']= faq_answ['prep5'].apply(remove_char)
faq_answ['prep7']= faq_answ['prep6'].apply(remove_duplicate)
faq_answ['prep8']= faq_answ['prep7'].apply(remove_stopwords)
faq_answ['lemma']= faq_answ['prep8'].apply(lemma)
faq_answ.head(10)

from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
faq['AnswersEncode'] = label.fit_transform(faq['Answers'])
faq

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

text = faq['Questions']
y= faq['AnswersEncode'].values


tfidf = TfidfVectorizer(use_idf=True, analyzer='word', stop_words='english', token_pattern=r'\b[^\d\W]+\b', ngram_range=(1,2))
X_train = tfidf.fit_transform(text)
print(X_train)


lsvc = LinearSVC(random_state = 2021)
lsvc.fit(X_train, y)


search_test = [
    "How can I recover?",
    "Is cannabis dangerous?",
    "What is the side effect of drinking?"
]

search_engine = tfidf.transform(search_test)
result = lsvc.predict(search_engine)

for question in result:
    faq_data = faq.loc[faq.isin([question]).any(axis=1)]
    print("Answer: ", faq_data['Answers'].values)


